In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cuda:1"
floattype = torch.float

batchsize = 512
nsamples = 8
npoints = 5
emsize = 512


class Graph_Transformer(nn.Module):
    def __init__(self, emsize = 64, nhead = 8, nhid = 1024, nlayers = 3, ndecoderlayers = 0, dropout = 0.3):
        super().__init__()
        self.emsize = emsize
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        encoder_layers = TransformerEncoderLayer(emsize, nhead, nhid, dropout = dropout)
        decoder_layers = TransformerDecoderLayer(emsize, nhead, nhid, dropout = dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.transformer_decoder = TransformerDecoder(decoder_layers, ndecoderlayers)
        self.encoder = nn.Linear(3, emsize)
        self.outputattention_query = nn.Linear(emsize, emsize, bias = False)
        self.outputattention_key = nn.Linear(emsize, emsize, bias = False)
        self.start_token = nn.Parameter(torch.randn([emsize], device = device))
    
    def generate_subsequent_mask(self, sz): #last dimension will be softmaxed over when adding to attention logits, if boolean the ones turn into -inf
        #mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        #mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        
        mask = torch.triu(torch.ones([sz, sz], dtype = torch.bool, device = device), diagonal = 1)
        return mask
    
    def encode(self, src): #src must be [batchsize * nsamples, npoints, 3]
        src = self.encoder(src).transpose(0, 1)
        output = self.transformer_encoder(src)
        return output #[npoints, batchsize * nsamples, emsize]
    
    def decode_next(self, memory, tgt, route_mask): #route mask is [batchsize * nsamples, npoints], both memory and tgt must have batchsize and nsamples in same dimension (the 1th one)
        npoints = memory.size(0)
        batchsize = tgt.size(1)
        """if I really wanted this to be efficient I'd only recompute the decoder for the last tgt, and just remebering what the others looked like from before (won't change due to mask)"""
        """have the option to freeze the autograd on all but the last part of tgt, although at the moment this is a very natural way to say: initial choices matter more"""
        tgt_mask = self.generate_subsequent_mask(tgt.size(0))
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[tgt, batchsize * nsamples, emsize]
        output_query = self.outputattention_query(memory).transpose(0, 1) #[batchsize * nsamples, npoints, emsize]
        output_key = self.outputattention_key(output[-1]) #[batchsize * nsamples, emsize]
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.unsqueeze(-1)).squeeze(-1) #[batchsize * nsamples, npoints], technically don't need to scale attention as we divide by variance next anyway
        output_attention_tanh = output_attention.tanh() #[batchsize * nsamples, npoints]
        
        #we clone the route_mask incase we want to backprop using it (else it was modified by inplace opporations)
        output_attention = output_attention.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        output_attention_tanh = output_attention_tanh.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        
        return output_attention_tanh, output_attention #[batchsize * nsamples, npoints]
    
    def calculate_logprob(self, memory, routes): #memory is [npoints, batchsize * nsamples, emsize], routes is [batchsize * nsamples, npoints - 4], rather than backproping the entire loop, this saves vram (and computation)
        npoints = memory.size(0)
        ninternalpoints = routes.size(1)
        bigbatchsize = memory.size(1)
        memory_ = memory.gather(0, routes.transpose(0, 1).unsqueeze(2).expand(-1, -1, self.emsize)) #[npoints - 4, batchsize * nsamples, emsize] reorder memory into order of routes
        tgt = torch.cat([self.start_token.unsqueeze(0).unsqueeze(1).expand(1, bigbatchsize, -1), memory_[:-1]]) #[npoints - 4, batchsize * nroutes, emsize], want to go from memory to tgt
        tgt_mask = self.generate_subsequent_mask(ninternalpoints)
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[npoints - 4, batchsize * nsamples, emsize]
        """want probability of going from key to query, but first need to normalise (softmax with mask)"""
        output_query = self.outputattention_query(memory_).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        output_key = self.outputattention_key(output).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        attention_mask = torch.full([ninternalpoints, ninternalpoints], True, device = device).triu(1) #[npoints - 4, npoints - 4], True for i < j
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.transpose(-1, -2))
        """quick fix to stop divergence"""
        output_attention_tanh = output_attention.tanh()
        
        output_attention_tanh = output_attention_tanh.masked_fill(attention_mask, float('-inf'))
        output_attention_tanh = output_attention_tanh - output_attention_tanh.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        output_attention = output_attention.masked_fill(attention_mask, float('-inf'))
        output_attention = output_attention - output_attention.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        """infact I'm almost tempted to not mask choosing a previous point, so it's forced to learn it and somehow incorporate it into its computation, but without much impact on reinforcing good examples"""
        logprob_tanh = output_attention_tanh.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        logprob = output_attention.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        return logprob_tanh, logprob #[batchsize * nsamples]

NN = Graph_Transformer().to(device)
optimizer = optim.Adam(NN.parameters())


class environment:    
    def reset(self, npoints, batchsize, nsamples=1, corner_points = None, initial_triangulation = None):
        """
        corner_points, etc., shoudn't include a batch dimension
        """
        if corner_points == None:
            ncornerpoints = 4
        else:
            ncornerpoints = corner_points.size(0)
        if npoints <= ncornerpoints:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = (
            batchsize * nsamples
        )  # so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = (
            torch.rand([batchsize, npoints - ncornerpoints, 3], dtype = floattype, device=device)
            .unsqueeze(1)
            .expand(-1, nsamples, -1, -1)
            .reshape(self.batchsize, npoints - ncornerpoints, 3)
        )
        if corner_points == None:
            self.corner_points = torch.tensor(
                [[0, 0, 0], [3, 0, 0], [0, 3, 0], [0, 0, 3]], dtype = floattype, device=device
            )
        else:
            self.corner_points = corner_points
        self.points = torch.cat(
            [
                self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1),
                self.points,
            ],
            dim=-2,
        )  # [batchsize * nsamples, npoints, 3]
        self.points_mask = torch.cat(
            [
                torch.ones([self.batchsize, ncornerpoints], dtype=torch.bool, device=device),
                torch.zeros(
                    [self.batchsize, npoints - ncornerpoints], dtype=torch.bool, device=device
                ),
            ],
            dim=1,
        )
        self.points_sequence = torch.empty(
            [self.batchsize, 0], dtype=torch.long, device=device
        )

        """
        points are now triples
        triangles are now quadruples
        edges are now still just indices, but there are four of them per 'triangle', and they correspond to triples of points, not pairs
        we use  0,2,1  0,3,2  0,1,3  1,2,3  as the order of the four 'edges'/faces
        opposite face is always ordered such that the last two indices are swapped
        faces are always read ANTICLOCKWISE
        
        first three points of tetrahedron MUST be read clockwise (from the outside) to get correct sign on incircle test
        
        new point will be inserted in zeroth position, so if corresponding face of REMOVED tetrahedron is [x,y,z] (being read anticlockwise from outside in) new tetrahedron is [p, x, y, z]
        """
        
        """
        number of tetrahedra is not the same for each batch (in 3D), so store as a big list, and remember batch index that it comes from
        """
        if corner_points == None:
            initial_triangulation = torch.tensor([[0, 1, 2, 3]], dtype=torch.long, device=device)
        
        self.partial_delaunay_triangles = initial_triangulation.unsqueeze(0).expand(self.batchsize, -1, -1).reshape(-1, 4)
        self.batch_index = torch.arange(self.batchsize, dtype = torch.long, device = device).unsqueeze(1).expand(-1, initial_triangulation.size(0)).reshape(-1)
        
        self.batch_triangles = self.partial_delaunay_triangles.size(0) #[0]
        self.ntriangles = torch.full([self.batchsize], initial_triangulation.size(0), dtype = torch.long, device = device) #[self.batchsize]
        
        self.cost = torch.zeros([self.batchsize], dtype = floattype, device=device)

        self.logprob = torch.zeros([self.batchsize], dtype = floattype, device=device, requires_grad=True)

    def update(self, point_index):  # point_index is [batchsize]
        
        assert point_index.size(0) == self.batchsize
        assert str(point_index.device) == device
        assert self.points_mask.gather(1, point_index.unsqueeze(1)).sum() == 0
        
        triangles_coordinates = self.points[self.batch_index.unsqueeze(1), self.partial_delaunay_triangles] # [batch_triangles, 4, 3]
        
        newpoint = self.points[self.batch_index, point_index[self.batch_index]] # [batch_triangles, 3]

        incircle_matrix = torch.cat(
            [
                newpoint.unsqueeze(1),
                triangles_coordinates,
            ],
            dim=-2,
        )  # [batch_triangles, 5, 3]
        incircle_matrix = torch.cat(
            [
                (incircle_matrix * incircle_matrix).sum(-1, keepdim=True),
                incircle_matrix,
                torch.ones([self.batch_triangles, 5, 1], dtype = floattype, device=device),
            ],
            dim=-1,
        )  # [batch_triangles, 5, 5]
        assert incircle_matrix.dtype == floattype
        assert str(incircle_matrix.device) == device
        
        incircle_test = (
            incircle_matrix.det() > 0
        )  # [batch_triangles], is True if inside incircle
        
        conflicts = incircle_test.sum()
        
        conflicting_triangles = self.partial_delaunay_triangles[incircle_test] # [conflicts, 4]
        
        conflicting_edges_index0 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([0, 0, 0, 1])
        conflicting_edges_index0 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index1 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([2, 3, 1, 2])
        conflicting_edges_index1 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index2 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([1, 2, 3, 3])
        conflicting_edges_index2 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges = torch.cat([conflicting_edges_index0.view(-1).unsqueeze(-1), conflicting_edges_index1.view(-1).unsqueeze(-1), conflicting_edges_index2.view(-1).unsqueeze(-1)], dim = -1).reshape(-1, 3) # [conflicts * 4, 3]
        
        edge_batch_index = self.batch_index[incircle_test].unsqueeze(1).expand(-1, 4).reshape(-1) # [conflicts * 4]
        
        indices = torch.LongTensor([0, 2, 1])
        comparison_edges = conflicting_edges[:, indices] # [conflicts * 4, 3]        
        
        unravel_nomatch_mask = torch.ones([conflicts * 4], dtype = torch.bool, device = device) # [conflicts * 4]
        i = 1
        while True:
            
            todo_mask = unravel_nomatch_mask[:-i].logical_and(edge_batch_index[:-i] == edge_batch_index[i:])
            if i % 4 == 0:
                if todo_mask.sum() == 0:
                    break
            
            match_mask = todo_mask.clone()
            match_mask[todo_mask] = (conflicting_edges[:-i][todo_mask] != comparison_edges[i:][todo_mask]).sum(-1).logical_not()
            
            unravel_nomatch_mask[:-i][match_mask] = False
            unravel_nomatch_mask[i:][match_mask] = False
            
            i += 1
        
        batch_newtriangles = unravel_nomatch_mask.sum()
        
        nomatch_edges = conflicting_edges[unravel_nomatch_mask] # [batch_newtriangles, 3], already in correct order to insert into 1,2,3 (since already anticlockwise from outside in)
        assert list(nomatch_edges.size()) == [batch_newtriangles, 3]
        nomatch_batch_index = edge_batch_index[unravel_nomatch_mask] # [batch_newtriangles]
        
        nomatch_newpoint = point_index[nomatch_batch_index] # [batch_newtriangles]
        
        newtriangles = torch.cat([nomatch_newpoint.unsqueeze(1), nomatch_edges], dim = -1) # [batch_newtriangles, 4]
        
        
        nremoved_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        nnew_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        
        indices = self.batch_index[incircle_test]
        nremoved_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        indices = edge_batch_index[unravel_nomatch_mask]
        nnew_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        assert (nnew_triangles <= 2 * nremoved_triangles + 2).logical_not().sum().logical_not()
        
        """
        NOTE:
        I THINK it's possible for nnew_triangles to be less than nremoved_triangles (or my code is just buggy...)
        """
        
        assert nnew_triangles.sum() == batch_newtriangles
        assert nremoved_triangles.sum() == incircle_test.sum()
        
        nadditional_triangles = nnew_triangles - nremoved_triangles # [batchsize]
        ntriangles = self.ntriangles + nadditional_triangles # [batchsize]
        
        partial_delaunay_triangles = torch.empty([ntriangles.sum(), 4], dtype = torch.long, device = device)
        batch_index = torch.empty([ntriangles.sum()], dtype = torch.long, device = device)
        
        cumulative_triangles = torch.cat([torch.zeros([1], dtype = torch.long, device = device), nnew_triangles.cumsum(0)[:-1]]) # [batchsize], cumulative sum starts at zero
        
        """
        since may actually have LESS triangles than previous round, we insert all that survive into the first slots (in that batch)
        """
        good_triangle_indices = torch.arange(incircle_test.logical_not().sum(), dtype = torch.long, device = device)
        good_triangle_indices += cumulative_triangles[self.batch_index[incircle_test.logical_not()]]
        bad_triangle_indices_mask = torch.ones([ntriangles.sum(0)], dtype = torch.bool, device = device)
        bad_triangle_indices_mask.scatter_(0, good_triangle_indices, False)
        
        assert good_triangle_indices.size(0) == incircle_test.logical_not().sum()
        assert bad_triangle_indices_mask.sum() == batch_newtriangles
        
        partial_delaunay_triangles[good_triangle_indices] = self.partial_delaunay_triangles[~incircle_test]
        batch_index[good_triangle_indices] = self.batch_index[~incircle_test]
        
        partial_delaunay_triangles[bad_triangle_indices_mask] = newtriangles
        batch_index[bad_triangle_indices_mask] = nomatch_batch_index
        
        self.partial_delaunay_triangles = partial_delaunay_triangles
        self.batch_index = batch_index
        
        self.ntriangles = ntriangles
        self.batch_triangles = self.partial_delaunay_triangles.size(0)
        
        self.points_mask.scatter_(
            1, point_index.unsqueeze(1).expand(-1, self.npoints), True
        )
        self.points_sequence = torch.cat(
            [self.points_sequence, point_index.unsqueeze(1)], dim=1
        )
        
        self.cost += nremoved_triangles
        return
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, npoints]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob = self.logprob + probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    

env = environment()


def train(epochs = 30000, npoints = 14, batchsize = 100, nsamples = 8):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(4, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        
        NN.eval()
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        fixed_baseline = 0.5 * torch.ones([1], device = device)
        min_baseline = env.cost.view(batchsize, nsamples)[:, :-1].min(-1)[0] #[batchsize], minimum cost
        baseline = greedy_baseline
        positive_reinforcement = - F.relu( - (env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))) #don't scale positive reinforcement
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        positive_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) <= -0.05
        negative_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) > 0.3
        """
        binary positive reinforcement
        """
        #loss = - ((logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement_binary).mean() #+ (logprob.view(batchsize, nsamples)[:, :-1] > -1) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement_binary
        """
        clipped binary reinforcement
        """
        loss = ( 
                - logprob.view(batchsize, nsamples)[:, :-1] 
                * (logprob.view(batchsize, nsamples)[:, :-1] < 0) 
                * positive_reinforcement_binary 
                + logprob.view(batchsize, nsamples)[:, :-1] 
                * (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 3) 
                * negative_reinforcement_binary 
        ).mean()
        """
        clipped binary postive, clipped weighted negative
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement ).mean()
        """
        clipped reinforcement without rescaling
        """
        #loss = ((logprob.view(batchsize, nsamples)[:, :-1] < -0.7) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement + (logprob.view(batchsize, nsamples)[:, :-1] > -5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement).mean()
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -3) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item())
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
        
        

In [ ]:
train(npoints = 9, batchsize = 4000, nsamples = 8) #greedyprob - 3, cutoff for negative is greedy performance + 0.3

14.853750228881836 -4.285572052001953 -4.752714157104492 -4.363935947418213 -5.3497443199157715 -5.3497443199157715
14.144750595092773 -4.0174736976623535 -4.691596984863281 -3.760115146636963 -5.109899044036865 -5.109898567199707
13.866500854492188 -3.3673412799835205 -4.405815601348877 -3.6860458850860596 -4.177791118621826 -4.177791595458984
13.704000473022461 -2.848935127258301 -4.061428070068359 -2.8104450702667236 -5.484803676605225 -5.484803676605225
13.769250869750977 -2.936551809310913 -4.079438209533691 -2.840942859649658 -2.5115230083465576 -2.5115227699279785
13.630500793457031 -2.5256431102752686 -3.728724956512451 -2.854498863220215 -5.507308006286621 -5.507308006286621
13.488750457763672 -2.2349767684936523 -3.448272943496704 -2.175503730773926 -2.2116546630859375 -2.2116549015045166
13.57175064086914 -2.1521031856536865 -3.374720335006714 -2.32255482673645 -3.423473358154297 -3.4234728813171387
13.410250663757324 -2.205720901489258 -3.450836658477783 -2.210513114929199 

12.51675033569336 -1.1364680528640747 -2.162637233734131 -0.8624849319458008 -2.7845511436462402 -2.7845516204833984
12.402750968933105 -1.1231200695037842 -2.1543545722961426 -0.25236931443214417 -0.8943203091621399 -0.8943204283714294
12.451000213623047 -1.1171259880065918 -2.118339776992798 -1.5806159973144531 -1.003399133682251 -1.003399133682251
12.4660005569458 -1.1112033128738403 -2.0917885303497314 -0.7521783113479614 -3.6681833267211914 -3.6681833267211914
12.431000709533691 -1.1129413843154907 -2.1265952587127686 -1.0975513458251953 -1.1057846546173096 -1.1057846546173096
12.499500274658203 -1.1096242666244507 -2.1254825592041016 -0.5369235277175903 -2.1756906509399414 -2.1756911277770996
12.46725082397461 -1.1042706966400146 -2.113657236099243 -0.6815836429595947 -0.5739336013793945 -0.5739336013793945
12.481000900268555 -1.111655354499817 -2.1005828380584717 -0.8924870491027832 -1.0281438827514648 -1.0281438827514648
12.474750518798828 -1.093450665473938 -2.0878851413726807

12.293500900268555 -0.9537290930747986 -1.8370583057403564 -0.48370206356048584 -3.012345314025879 -3.0123450756073
12.274250984191895 -0.9410200715065002 -1.8363938331604004 -1.2239837646484375 -1.7273870706558228 -1.727386713027954
12.239250183105469 -0.9260239601135254 -1.8072729110717773 -1.3675873279571533 -4.175751686096191 -4.175751686096191
12.237000465393066 -0.9316747188568115 -1.8191933631896973 -0.9736325144767761 -1.8897340297698975 -1.8897343873977661
12.288500785827637 -0.9373863935470581 -1.788427710533142 -0.1218264102935791 -1.077492594718933 -1.077492594718933
12.247000694274902 -0.9203511476516724 -1.798329472541809 -1.8600391149520874 -1.5025877952575684 -1.5025877952575684
12.330000877380371 -0.9201937913894653 -1.7933919429779053 -1.257981777191162 -0.374561071395874 -0.374561071395874
12.28950023651123 -0.9057944416999817 -1.7781617641448975 -0.6874027252197266 -4.793403625488281 -4.793403625488281
12.276750564575195 -0.9121960997581482 -1.774053931236267 -0.740

12.223750114440918 -0.8878887295722961 -1.7068986892700195 -1.4307608604431152 -1.33298659324646 -1.33298659324646
12.227750778198242 -0.8684875965118408 -1.67384934425354 -0.591294527053833 -1.00201416015625 -1.00201416015625
12.171751022338867 -0.861501157283783 -1.6934326887130737 -1.35396409034729 -2.9602065086364746 -2.9602065086364746
12.230250358581543 -0.8560959696769714 -1.6796261072158813 -1.0990254878997803 -2.80643630027771 -2.8064358234405518
12.179000854492188 -0.865044116973877 -1.6592379808425903 -0.3385345935821533 -1.6248321533203125 -1.624832272529602
12.193750381469727 -0.8437201380729675 -1.6546720266342163 -0.2691841125488281 -4.805098056793213 -4.805098533630371
12.270500183105469 -0.8554201126098633 -1.659889578819275 -0.792266845703125 -0.594912052154541 -0.594912052154541
12.190500259399414 -0.8800803422927856 -1.6911211013793945 -0.6154090166091919 -0.18309921026229858 -0.18309921026229858
12.152750968933105 -0.8662168383598328 -1.6888575553894043 -1.90727472

12.061500549316406 -0.8338003158569336 -1.6484975814819336 -0.32158076763153076 -1.6754019260406494 -1.675402045249939
12.141250610351562 -0.850416898727417 -1.6504286527633667 -1.412947177886963 -2.612771987915039 -2.61277174949646
12.140250205993652 -0.8331663012504578 -1.618894100189209 -1.5442850589752197 -0.3705143928527832 -0.3705143928527832
12.144001007080078 -0.8276548981666565 -1.604682207107544 -0.7719029188156128 -0.2902333736419678 -0.2902334928512573
12.167750358581543 -0.8262561559677124 -1.592577576637268 -0.6181134581565857 -1.3269120454788208 -1.3269121646881104
12.1535005569458 -0.8159844279289246 -1.5759302377700806 -0.30882108211517334 -0.33037352561950684 -0.33037352561950684
12.132000923156738 -0.8093470335006714 -1.5615814924240112 -0.29311835765838623 -2.0919151306152344 -2.0919151306152344
12.14275074005127 -0.8133511543273926 -1.5818389654159546 -0.729451596736908 -1.884695291519165 -1.884695291519165
12.120500564575195 -0.807123064994812 -1.5856993198394775 

...

11.542250633239746 -0.4459170401096344 -0.8988454937934875 -0.424884557723999 -0.4341285228729248 -0.434128999710083
11.546000480651855 -0.43518587946891785 -0.9157872200012207 -0.952276349067688 -0.31892454624176025 -0.31892454624176025
11.580500602722168 -0.45191067457199097 -0.9193589687347412 -0.8548785448074341 -0.20563268661499023 -0.20563268661499023
11.526750564575195 -0.4383344352245331 -0.8808954954147339 -0.29276394844055176 -1.2580260038375854 -1.2580257654190063
11.540000915527344 -0.44034919142723083 -0.886707067489624 -0.6106510162353516 -5.028844356536865 -5.028843879699707
11.55875015258789 -0.44215017557144165 -0.9099684357643127 -0.5492153167724609 -6.0268874168396 -6.026887893676758
11.542750358581543 -0.4358792006969452 -0.9118344783782959 -0.6389250755310059 -2.024542808532715 -2.024543285369873
11.531250953674316 -0.45792725682258606 -0.8918505907058716 -0.9534122943878174 -0.8804492950439453 -0.8804492950439453
11.564250946044922 -0.4205094277858734 -0.887680232

11.607500076293945 -0.45765385031700134 -0.9050087332725525 -0.8405938148498535 -0.10019350051879883 -0.10019350051879883
11.587250709533691 -0.45164743065834045 -0.8948992490768433 -0.6986854076385498 -1.172501802444458 -1.172502040863037
11.559000968933105 -0.43484407663345337 -0.8931317329406738 -0.4856797456741333 -0.05316448211669922 -0.05316448211669922
11.550251007080078 -0.44566699862480164 -0.8920820355415344 -1.6228079795837402 -1.1972737312316895 -1.1972737312316895
11.536250114440918 -0.44682759046554565 -0.9043218493461609 -0.26346349716186523 -0.9956934452056885 -0.9956939220428467
11.6222505569458 -0.44721296429634094 -0.9059395790100098 -0.37068647146224976 -1.5946980714797974 -1.59469735622406
11.595500946044922 -0.44501420855522156 -0.9077239036560059 -0.5630149841308594 -0.0946175754070282 -0.09461709856987
11.542250633239746 -0.45645350217819214 -0.9090607166290283 -0.4842805862426758 -0.9306361675262451 -0.930635929107666
11.544000625610352 -0.4531174898147583 -0.9

11.580500602722168 -0.43551257252693176 -0.8773455023765564 -5.996227264404297e-05 -0.5824165344238281 -0.5824170112609863
11.571250915527344 -0.4394669234752655 -0.8643355369567871 -0.03628087043762207 -1.4376428127288818 -1.4376425743103027
11.585750579833984 -0.42913928627967834 -0.8693293929100037 -1.1294362545013428 -0.018395423889160156 -0.018395423889160156
11.521750450134277 -0.4254951477050781 -0.8713486194610596 -0.32039976119995117 -0.12039637565612793 -0.12039637565612793
11.577250480651855 -0.43767058849334717 -0.8740950226783752 -0.07963752746582031 -0.05811309814453125 -0.05811309814453125
11.57075023651123 -0.43465420603752136 -0.8795207142829895 -1.0141119956970215 -2.555959701538086 -2.5559606552124023
11.595000267028809 -0.4337986707687378 -0.8749988675117493 -0.989081859588623 -0.38661766052246094 -0.38661766052246094
11.53950023651123 -0.41645777225494385 -0.8642054796218872 -0.45920705795288086 -0.15391579270362854 -0.15391582250595093
11.55525016784668 -0.4222703

11.579500198364258 -0.4250107705593109 -0.8508063554763794 -0.7033443450927734 -0.10922831296920776 -0.10922831296920776
11.56725025177002 -0.4295766055583954 -0.8595152497291565 -0.3603322505950928 -0.2241525650024414 -0.2241525650024414
11.540250778198242 -0.42044898867607117 -0.8516154885292053 -0.543632984161377 -1.8024749755859375 -1.8024754524230957
11.540250778198242 -0.4377850592136383 -0.895397424697876 -0.6856045722961426 -0.051515817642211914 -0.051515817642211914
11.581501007080078 -0.4452191889286041 -0.9058681726455688 -0.14084410667419434 -0.8648636341094971 -0.8648636341094971
11.570250511169434 -0.4415687918663025 -0.9050098657608032 -1.0801360607147217 -9.116422653198242 -9.116421699523926
11.518750190734863 -0.45146995782852173 -0.9125153422355652 -0.881880521774292 -0.3039734363555908 -0.3039734363555908
11.580500602722168 -0.4525322914123535 -0.9023833870887756 -0.33107733726501465 -0.0026803016662597656 -0.0026803016662597656
11.584250450134277 -0.446137398481369 

11.635250091552734 -0.4238547384738922 -0.846809983253479 -0.06891512870788574 -5.866019248962402 -5.866018772125244
11.55150032043457 -0.4268149733543396 -0.852449893951416 -0.5525722503662109 -3.0272374153137207 -3.0272393226623535
11.539250373840332 -0.4137563705444336 -0.8460262417793274 -0.8209242820739746 -0.13531029224395752 -0.13531029224395752
11.562250137329102 -0.4262579679489136 -0.8720444440841675 -0.7672042846679688 -0.025232791900634766 -0.025232791900634766
11.60575008392334 -0.42600613832473755 -0.8752848505973816 -0.004384040832519531 -0.37148141860961914 -0.37148141860961914
11.586750984191895 -0.42123943567276 -0.8815839290618896 -0.0028371810913085938 -0.13742542266845703 -0.13742542266845703
11.557750701904297 -0.4431719481945038 -0.8791083693504333 -0.629603385925293 -1.1889266967773438 -1.1889268159866333
11.583000183105469 -0.44274765253067017 -0.9117444157600403 -1.1024537086486816 -0.03181743621826172 -0.03181791305541992
11.597500801086426 -0.443979322910308